In [5]:
import pandas as pd
from bs4 import BeautifulSoup
import requests
import re

In [21]:
df = pd.read_excel("data/santafe_2008-2011.xlsx")
df.head()

,title,date,link,media
0,Cuenca habla de un programa integral para la s...,2007-11-03 00:00:00,https://www.ellitoral.com/edicion-online/cuenc...,ellitoral
1,No prevén cambios inmediatos en las jefaturas ...,2007-12-02 00:00:00,https://www.ellitoral.com/edicion-online/preve...,ellitoral
2,En un accidente murió el jefe de Drogas Peligr...,2007-12-30 00:00:00,https://www.ellitoral.com/edicion-online/accid...,ellitoral
3,Encontraron otros diez kilos de marihuana en P...,2008-09-24 00:00:00,https://www.ellitoral.com/edicion-online/encon...,ellitoral
4,Con la orden de “caiga quien caiga”,2009-01-01 00:00:00,https://www.ellitoral.com/index.php/diarios/20...,ellitoral


In [30]:

def content_extract(url):
    try:
        response = requests.get(url)
        response.raise_for_status()  # Raise an HTTPError for bad responses
        

        
        soup = BeautifulSoup(response.text, 'html.parser', from_encoding="utf-8")
        content = soup.findAll('p')
        content_list = [p.get_text(strip=True) for p in content]
        
        # Apply rules based on text in the URL
        if 'lacapital.com' in url:
            content_list = content_list[1:-1]  # Discard the first and last
        elif 'pagina12.com' in url:
            content_list = content_list[3:-2]  # Discard the first 3 and last 2
        elif 'elciudadanoweb.com' in url:
            content_list = content_list[:-2]  # Discard the last 2
        
        # Return the processed list or a default message
        return content_list if content_list else ["No content found"]
    except requests.exceptions.RequestException as e:
        return [f"Request error: {e}"]
    except Exception as e:
        return [f"Error: {e}"]

# Apply the function to the 'links' column and create a new column 'content'
df['content'] = df['link'].apply(content_extract)


c:\Users\patricio\.pyenv-win-venv\envs\rag_research\lib\site-packages\bs4\__init__.py:228: UserWarning: You provided Unicode markup but also provided a value for from_encoding. Your from_encoding will be ignored.
  warnings.warn("You provided Unicode markup but also provided a value for from_encoding. Your from_encoding will be ignored.")


In [ ]:
df

In [29]:
ex = "https://www.ellitoral.com/politica/santa-fe-pedira-coordinar-acciones-inseguridad-nacion_0_WIbQaUQCQI.html"

response = requests.get(ex)
print(response.encoding)
        
"""        # Handle encoding explicitly
        if response.encoding is None:
            response.encoding = response.apparent_encoding  # Detect encoding if not provided"""

utf-8


'        # Handle encoding explicitly\n        if response.encoding is None:\n            response.encoding = response.apparent_encoding  # Detect encoding if not provided'

In [33]:
import unicodedata

# Recode text function
def recode_text(text_list):
    def fix_encoding(text):
        try:
            return unicodedata.normalize("NFKC", text.encode("latin1").decode("utf-8"))
        except (UnicodeEncodeError, UnicodeDecodeError):
            return text  # Return original text if encoding fails
    
    cleaned_text = [
        fix_encoding(item).strip() 
        for item in text_list 
        if item.strip()  # Remove empty strings
    ]
    return cleaned_text

# Apply function to rows with "rosario12" in "media"
def clean_content_column(df):
    df.loc[df['media'] == 'rosario12', 'content'] = df.loc[df['media'] == 'rosario12', 'content'].apply(recode_text)
    return df


# Clean DataFrame
df_cleaned = clean_content_column(df)


In [37]:
def replace_newline_and_slash_in_list(text_list):
    """
    Reemplaza '\n' y '/' por un espacio en cada string de una lista.

    Args:
        text_list (list): Lista de strings en la que se reemplazarán los caracteres.

    Returns:
        list: Lista con los caracteres reemplazados en cada string.
    """
    if isinstance(text_list, list):  # Verifica que sea una lista
        return [text.replace("\n", " ").replace("/", " ") for text in text_list]
    return text_list  # Devuelve sin cambios si no es una lista


df_cleaned["content"] = df_cleaned["content"].apply(replace_newline_and_slash_in_list)

In [38]:
df_cleaned.to_excel("data/scrapped_content.xlsx")